# Identifying patterns in protein adaptation to temperature.

**Acknowledgement**: A huge thanks to [Antonin Affholder](https://www.ens.psl.eu/actualites/antonin-affholder) for the dataset and global layout of this mini-project.

## Problem

<!-- Proteins of species living in high or low temperature have specific properties. -->

Proteins from bacteria and archaea have often been used to elucidate patterns in protein adaptation across wide temperature ranges. To identify these patterns, researchers compared sequences of families of homologous high and low-temperature proteins. This approach allows to extract recurring amino acid replacement trends potentially important for thermal adaptation (e.g. more charged and hydrophobic amino acids).

From a set of protein sequences of mesophilic* and hyperthermophilic** archaea, your mission, should you choose to accept it, is to code a script that would detect preferential amino acid replacement in hyperthermophilic species, that would indicate adaptation to high temperature.
- `*` mesophilic organism = organism that grows in moderate temperatures (between 20°C and 45°C)
- `**`hyperthermophilic organism = organism that thrives in extremely hot environments (above 60°C)

Here, we are interested in comparing hyperthermophiles with mesophiles.

### Details

The dataset you will use (`Ftr_A.aln`) consists in an alignement of the set of protein sequences (an enzyme involved in methanogenesis) of mesophilic and hyperthermophilic archaea species, aligned with the software MAFFT, it follows the same standards as the fasta format.
The `metadata.csv` file is a table containing the information on the dataset and most importantly associates sequences ID with organism type (hyperthermophile, thermophile or mesophile).

The number of substitutions from mesophilic sequences to hyperthermophilic sequences will be stored in a matrix. This will be a 20x20 matrix (as there are 20 amino acids). Element M[i,j] will be the number of substitutions from amino-acid i in the mesophilic sequences to amino-acid j in the hyperthermophile sequences.

The substitution ratios are then computed by dividing the substitution matrix M by its transpose.

<!-- Is there a preferential amino acid replacement in hyperthermophilic species? Can you identify groups of amino acids (charged, polar, hydrophobic) that are preferentially replaced? -->

### Some tips

To do so, you will need to:

- Find a way to read data
- Classify the two types of sequences (mesophilic and hyperthermophilic)
- Compute and store in a matrix the number of amino acids substitutions between the two sets of sequences (mesophilic and hyperthermophilic)
- Compute the substitution ratios from mesophilic to hyperthermophilic sequences
- Visually represent the substitution matrix as a heatmap
- Conclude on a potential preferential amino acid replacement 


In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

## Step 1 - File reading
### Metadata file

In [ ]:
# TODO: Load metadata.csv as a dataframe
metadata = ...
metadata.head(5)

In [ ]:
# TODO: Find the organism IDs of the mesophilic and hyperthermophilic archaea

# hyperthermophilic
ht_id = ...
# mesophilic
t_id =  ...

print(ht_id)
print(t_id)
# We expect:
# [ 190192  880724  243232  573063  573064  523846  579137 1069083]
# [ 647171  187420  523845   79929  647113  523844 1041930  882090  419665
#   351160 1715806  323259    2162]

### Aligned sequences
#### Reading of a row of data

In [ ]:
def read_aln_row(row):
    """Parse a row of an aln file and return its content.
    
    If the row does not contain any data, None is returned.
    
    Parameters
    ----------
    row: str
        A row of an aln file.
        
    Returns
    -------
    parsed_row: dict or None
        A dictionary containing the parsed data.
        The dictionary has format:
        
            {"organism_id": organism_id,
             "sequence": sequence,
              "end_position": end_position}
              
        Where organism_id, sequence and end_position correspond to:
        
        organism_id: int
            The organism ID.

        sequence: str
            The protein sequence stored on the row.

        end_position: int
            The position of the last amino-acid of the row.
    """
    # TODO

In [ ]:
# Example row
row = "190192    MEINGVEIEDTFAEAFEAKMARVLITAASHKWAMIAVKEATGFGTSVIMCPAEAGIDCYVPPEETPDGRP 70\n"

read_aln_row(row)
# We expect:
# {'organism_id': 190192,
#  'sequence': 'MEINGVEIEDTFAEAFEAKMARVLITAASHKWAMIAVKEATGFGTSVIMCPAEAGIDCYVPPEETPDGRP',
#  'end_position': 70}
# WARNING: organism_id and end_position should be integers

#### Parsing of the full file

In [ ]:
def read_text_file(path):
    """Return the rows of a text file as elements of a list
    
    Parameters
    ----------
    path: str
        The path to the text file to read
    
    Returns
    -------
    list of str
        A list containing the text file's rows.
    """
    # TODO

In [ ]:
# Test it on the data
data_path = "Ftr_A.aln"
# Print the first four lines of the file
read_text_file(data_path)[:4]

# We expect:
# ['CLUSTAL W 2.0 multiple sequence alignment\n',
#  '\n',
#  '190192    MEINGVEIEDTFAEAFEAKMARVLITAASHKWAMIAVKEATGFGTSVIMCPAEAGIDCYVPPEETPDGRP 70\n',
#  '243232    MEINGVYIEDTFAEAFPIWVSRVLITAATKKWAKIAATEATGFGCSVIMCPAEAGIEKYVPPSKTPDGRP 70\n']

In [ ]:
def load_aln_data(path):
    """Load an aln file as a pandas dataframe
    
    Parameters
    ----------
    path: str
        The path to the data file.
        
    Returns
    -------
    df: pd.DataFrame
        The aligned sequences stored in a dataframe.
        Each row correspond to a position in the sequence,
        each column to an organism ID.
    """
    # Open the file whose path is given as input and read its content
    row_all = read_text_file(path)
    
    # Parse each row and store those containing data in a list
    parsed_row_all = []
    for row in row_all:
        parsed_row = read_aln_row(row)
        if parsed_row is not None:
            parsed_row_all.append(parsed_row)

    # Find the maximum value of end position and all organism IDs
    # TODO
    end_position_max = ...
    organism_id_all = ...
            
    # Preallocate the output dataframe
    df = pd.DataFrame(index=np.arange(1, end_position_max + 1),
                      columns=organism_id_all, dtype=str)
    
    # Store the data from each parsed row in df
    for parsed_row in parsed_row_all:
        # TODO

    return df

In [ ]:
data_path = "Ftr_A.aln"
data = load_aln_data(data_path)
data.head(5)

# We expect something similar to:
#    190192  243232  523846  573063  573064  579137  880724  1069083   2162  ...   
# 1       M       M       M       M       M       M       M       M       M   
# 2       E       E       K       E       E       E       E       E       E   
# 3       I       I       V       I       I       I       I       I       I   
# 4       N       N       N       N       N       N       N       N       N   
# 5       G       G       G       G       G       G       G       G       G   

## Step 2 - Number of substitutions
We will loop over each pair of mesophilic and hyperthermophilic organism and each position and store the comparison between each resulting pair of aminoacid in a pandas dataframe. 

In [ ]:
amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
               'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

substitution_count = pd.DataFrame(np.zeros((len(amino_acids), len(amino_acids)), dtype=int),
                                  index=amino_acids, columns=amino_acids)

print(t_id)  # mesophilic
print(ht_id) # hyperthermophilic


# TODO: for each mesophilic archae, loop over all hyperthermophilic organisms
# and over each position in the sequence and increment the count of substitutions
# E.G. substitution_count["A"]["L"] is how many time an Alanine in the sequence of a
# mesophilic archae was replaced by a leucine in the protein sequence of a hyperthermophilic
# archae
# TODO
substitution_count

### Step 4 - Substitution ration
Compute the substitution ratios by dividing the substitution matrix by its transpose and plot the result.

In [ ]:
# TODO: Compute the substitution ratios
substitution_ratios = ...

In [ ]:
substitution_ratios
# We expect something similar to:
#           A         C         D         E         F         G          H ...
# A  1.000000  4.050000  0.363636  5.125000  0.000000  1.118110        inf   
# C  0.246914  1.000000       NaN       NaN       NaN  0.590909        inf   
# D  2.750000       NaN  1.000000  1.314607       NaN  1.294118   0.000000   
# E  0.195122       NaN  0.760684  1.000000  0.000000  2.421053   3.310345   
# ...

In [ ]:
# TODO: Plot the output